# Instructions to Run the Deep Research Agent

To run this notebook successfully, you need to:

1. Make sure you have installed the required packages: `azure-ai-projects`, `azure-identity`, and `azure-ai-agents`
2. Update the environment variables in the second cell with your actual Azure resource information:
   - `PROJECT_ENDPOINT`: The endpoint URL for your Azure AI Projects
   - `BING_RESOURCE_NAME`: The name of your Azure Bing Search resource
   - `DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME`: The deployment name of your Deep Research model
   - `MODEL_DEPLOYMENT_NAME`: The deployment name of your base model

3. Run each cell sequentially

The notebook will:
- Create a research-output folder to store results
- Initialize a Deep Research agent
- Query for information about "retail adoption trends in Australia"
- Generate a research summary in the research-output folder

After running, check the research-output folder for the generated research summary.

In [ ]:
import os

# Create research-output directory
output_dir = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'research-output')
os.makedirs(output_dir, exist_ok=True)
print(f"Created output directory: {output_dir}")


In [ ]:
import os, time
from typing import Optional
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import DeepResearchTool, MessageRole, ThreadMessage

# Define the environment variables (replace with your actual Azure resource values)
os.environ["PROJECT_ENDPOINT"] = ""  # Your Azure AI Projects endpoint
os.environ["BING_RESOURCE_NAME"] = ""  # Your Bing resource name
os.environ["DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME"] = "o3-deep-research"  # Your Deep Research model deployment name
os.environ["MODEL_DEPLOYMENT_NAME"] = "gpt-4.1"  # Your base model deployment name

# Helper to fetch and print new agent responses
def fetch_and_print_new_agent_response(
    thread_id: str,
    agents_client: AgentsClient,
    last_message_id: Optional[str] = None,
) -> Optional[str]:
    response = agents_client.messages.get_last_message_by_role(
        thread_id=thread_id,
        role=MessageRole.AGENT,
    )
    if not response or response.id == last_message_id:
        return last_message_id
    print("\nAgent response:")
    print("\n".join(t.text.value for t in response.text_messages))
    for ann in response.url_citation_annotations:
        print(f"URL Citation: [{ann.url_citation.title}]({ann.url_citation.url})")
    return response.id

# Helper to write research summary into research-output folder
def create_research_summary(
        message: ThreadMessage,
        filepath: str = os.path.join('research-output', 'research_summary.md')
) -> None:
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    if not message:
        print("No message content provided, cannot create research summary.")
        return
    with open(filepath, 'w', encoding='utf-8') as fp:
        text_summary = '\n\n'.join([t.text.value.strip() for t in message.text_messages])
        fp.write(text_summary)
        if message.url_citation_annotations:
            fp.write('\n\n## References\n')
            seen = set()
            for ann in message.url_citation_annotations:
                url = ann.url_citation.url
                title = ann.url_citation.title or url
                if url not in seen:
                    fp.write(f"- [{title}]({url})\n")
                    seen.add(url)
    print(f"Research summary written to '{filepath}'")

# Initialize Azure clients
project_client = AIProjectClient(
    endpoint=os.environ['PROJECT_ENDPOINT'],
    credential=DefaultAzureCredential(),
)
conn_id = project_client.connections.get(name=os.environ['BING_RESOURCE_NAME']).id

deep_research_tool = DeepResearchTool(
    bing_grounding_connection_id=conn_id,
    deep_research_model=os.environ['DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME'],
)

with project_client:
    with project_client.agents as agents_client:
        agent = agents_client.create_agent(
            model=os.environ['MODEL_DEPLOYMENT_NAME'],
            name='my-agent',
            instructions='You are a helpful Agent that assists in researching market topics.',
            tools=deep_research_tool.definitions,
        )
        print(f"Created agent, ID: {agent.id}")
        thread = agents_client.threads.create()
        print(f"Created thread, ID: {thread.id}")

        # Send initial research prompt
        initial_query = 'Give me the latest research on retail adoption trends in Australia.'
        message = agents_client.messages.create(
            thread_id=thread.id,
            role='user',
            content=initial_query,
        )
        print(f"Created message, ID: {message.id}")
        last_id = None

        # 🔧 keep looping until the agent has enough info
        while True:
            run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)

            # poll the run
            while run.status in ("queued", "in_progress"):
                time.sleep(1)
                run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
                last_id = fetch_and_print_new_agent_response(thread.id, agents_client, last_id)
                print(f"Run status: {run.status}")

            if run.status == "failed":
                print(f"Run failed: {run.last_error}")
                break

            latest = agents_client.messages.get_last_message_by_role(
                thread_id=thread.id, role=MessageRole.AGENT
            )
            text = "\n".join(t.text.value for t in latest.text_messages).strip()

            if "?" in text:
                print("\nAgent asked for clarification:\n")
                print(text)

                follow_up = input("\nYour answer ➜ ").strip()
                if not follow_up:
                    print("No answer provided – stopping.")
                    break

                agents_client.messages.create(
                    thread_id=thread.id,
                    role="user",
                    content=follow_up,
                )
                continue


            # Otherwise the agent has produced its final answer
            create_research_summary(latest, os.path.join("research-output", "research_summary.md"))
            print(f"Run finished with status: {run.status}, ID: {run.id}")
            break  # exit main loop – research is complete

        agents_client.delete_agent(agent.id)
        print("Deleted agent")
